In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import gc
from tensorflow.keras.layers import Input
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.python.keras.engine import training
from tensorflow.python.framework.ops import Tensor
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, LeakyReLU, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.models import Sequential, Model
import optimise_util as opt
import audio_tagging_utils as utils

In [2]:
traindf=pd.read_csv(os.path.join('meta', 'train.csv'))
testdf=pd.read_csv(os.path.join('meta', 'test.csv'))
traindf["fname"]= traindf["fname"].apply(utils.append_ext)
testdf["fname"]= testdf["fname"].apply(utils.append_ext)

In [3]:
def kaggle_2d_conv_pool_cnn(model_input: Tensor, params: dict) -> training.Model:
    x = Conv2D(32, (3, 3), padding='same')(model_input)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][0])(x)
    
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][1])(x)
    
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(params["dropouts"][2])(x)
    
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(params["dropouts"][3])(x)
    x = Dense(41, activation='softmax')(x)

    model = Model(model_input, x, name='conv_pool_cnn')
    model.compile(optimizers.Adam(params["lr"][0]),loss="categorical_crossentropy",metrics=['accuracy'])
    
    return model

In [4]:
model_input = Input(shape=(64, 64, 3))

#Initialise the parameters
params = {} 
params["dropouts"] = [0.1, 0.1, 0.1, 0.1] 
params["lr"] = [0.001]

kaggle_2d_model = kaggle_2d_conv_pool_cnn(model_input, params)

#Calls utilities file, returns doct of optimised parameters
params = opt.optimise(kaggle_2d_model, traindf, params)

Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
Found 7578 validated image filenames belonging to 41 classes.
Found 1895 validated image filenames belonging to 41 classes.
Found 7579 validated image filenames belonging to 41 classes.
Found 1894 validated image filenames belonging to 41 classes.
Found 7579 validated image filenames belonging to 41 classes.
Found 1894 validated image filenames belonging to 41 classes.
Optimising LR, LR = 0.001

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 236 steps, validate for 59 steps
  1/236 [..............................] - ETA: 11:41

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv_pool_cnn/conv2d/Conv2D (defined at C:\Users\bacci\git\Audio-Tagging---Deep-Learning-Project\optimise_util.py:57) ]] [Op:__inference_distributed_function_1556]

Function call stack:
distributed_function


In [ ]:
params